In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

In [10]:
credits_df = pd.read_csv('../data/movie-set-data/credits.csv')
keywords_df = pd.read_csv('../data/movie-set-data/keywords.csv')
links_df = pd.read_csv('../data/movie-set-data/links.csv')
links_small_df = pd.read_csv('../data/movie-set-data/links_small.csv')
movie_meta_df = pd.read_csv('../data/movie-set-data/movies_metadata.csv')
ratings_df = pd.read_csv('../data/movie-set-data/ratings.csv')
ratings_small_df = pd.read_csv('../data/movie-set-data/ratings_small.csv')

/Users/ivan/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
